This IPYNB file was generated automatically by MOA GUI.<br>
Task name: moa.tasks.EvaluatePrequentialDelayedCV<br>
## Libraries importing


In [1]:
%maven nz.ac.waikato.cms.moa:moa:2019.05.0
%classpath "moa-flow-core.jar"
import moa.classifiers.meta.AdaptiveRandomForest;
import moa.streams.generators.RandomTreeGenerator;
import moa.evaluation.WindowClassificationPerformanceEvaluator;
import moa.evaluation.LearningEvaluation;
import moa.evaluation.preview.LearningCurve;
import moa.core.TimingUtils;
import moa.tasks.MainTask;
import com.yahoo.labs.samoa.instances.Instance;
import moa.core.Example;
import moa.core.Measurement;
import java.util.Random;
import java.util.LinkedList;
import moaflow.sink.DrawTable;
import moa.core.MiscUtils;


## Configuring learner, stream and evaluator


In [2]:
String learnerString = "meta.AdaptiveRandomForest -l (ARFHoeffdingTree -e 2000000 -g 75 -s GiniSplitCriterion -c 0.01 -l MC) -o (Percentage (M * (m / 100))) -m 80";
int randomSeed =1;
Random random = new Random(randomSeed);
int numFolds = 10;
AdaptiveRandomForest[] adapArray = new AdaptiveRandomForest[numFolds];
AdaptiveRandomForest adap = new AdaptiveRandomForest();adap.getOptions().setViaCLIString("-l (ARFHoeffdingTree -e 2000000 -g 75 -s GiniSplitCriterion -c 0.01 -l MC) -o (Percentage (M * (m / 100))) -m 80");
adap.prepareForUse();
adap.resetLearning();

String streamString = "generators.RandomTreeGenerator";
RandomTreeGenerator rand = new RandomTreeGenerator();
rand.prepareForUse();
adap.setModelContext(rand.getHeader());

String evaluatorString = "WindowClassificationPerformanceEvaluator";
WindowClassificationPerformanceEvaluator[] windArray = new WindowClassificationPerformanceEvaluator[numFolds];
WindowClassificationPerformanceEvaluator wind = new WindowClassificationPerformanceEvaluator();
for (int i = 0; i <adapArray.length; i++) {
    adapArray[i] = (AdaptiveRandomForest) adap.copy();
    adapArray[i].setModelContext(rand.getHeader());
    windArray[i] = (WindowClassificationPerformanceEvaluator) wind.copy();
}


## Setting environmental parameters


In [3]:
int maxInstances = 100000;
long instancesProcessed = 0;
int maxSeconds = -1;
int secondsElapsed = 0;
LearningCurve learningCurve = new LearningCurve("learning evaluation instances");
File dumpFile = new File("null");
PrintStream immediateResultStream = null;
if (dumpFile != null) {
    try {
        if (dumpFile.exists()) {
            immediateResultStream = new PrintStream(
                    new FileOutputStream(dumpFile, true), true);
        } else {
            immediateResultStream = new PrintStream(
                    new FileOutputStream(dumpFile), true);
        }
    } catch (Exception ex) {
        throw new RuntimeException(
                "Unable to open immediate result file: "  + dumpFile, ex);
    }
}
LinkedList<LinkedList<Example>> trainInstances = new LinkedList<LinkedList<Example>>();
for(int i = 0; i < adapArray.length; i++) {
   trainInstances.add(new LinkedList<Example>());
}
boolean firstDump = true;
boolean firstPrint = true;
boolean preciseCPUTiming = TimingUtils.enablePreciseTiming();
long evaluateStartTime = TimingUtils.getNanoCPUTimeOfCurrentThread();
long lastEvaluateStartTime = evaluateStartTime;
double RAMHours = 0.0;


## Testing & training, exporting result


In [4]:
DrawTable table = new DrawTable();
System.out.println( "Evaluating learner...");
while (rand.hasMoreInstances()
        && ((maxInstances < 0) || (instancesProcessed < maxInstances))
        && ((maxSeconds < 0) || (secondsElapsed < maxSeconds))) {
    instancesProcessed++;
    Example trainInst = rand.nextInstance();
    Example testInst = trainInst; //.copy();
    for(int i = 0; i < adapArray.length; i++) {
        double[] prediction = adapArray[i].getVotesForInstance(testInst);
        windArray[i].addResult(testInst, prediction);
        int k = 1;
        int validationMethodology = 0;
        switch (validationMethodology) {
            case 0: //Cross-Validation;
                k = instancesProcessed % adapArray.length == i ? 0: 1; //Test all except one
                break;
            case 1: //Bootstrap;
                k = MiscUtils.poisson(1, random);
                break;
            case 2: //Split-Validation;
                k = instancesProcessed % adapArray.length == i ? 1: 0; //Test only one
                break;
        }
        if (k > 0) {
            trainInstances.get(i).addLast(trainInst);
        }
        int delayLength = 1000;
        if(delayLength < trainInstances.get(i).size()) {
        Example trainInstI = trainInstances.get(i).removeFirst();
        adapArray[i].trainOnInstance(trainInstI);
        }
    }
    if (instancesProcessed % 10000 == 0
            ||  rand.hasMoreInstances() == false) {
        long evaluateTime = TimingUtils.getNanoCPUTimeOfCurrentThread();
        double time = TimingUtils.nanoTimeToSeconds(evaluateTime - evaluateStartTime);
        double timeIncrement = TimingUtils.nanoTimeToSeconds(evaluateTime - lastEvaluateStartTime);
        for (int i = 0; i <adapArray.length; i++) {
        double RAMHoursIncrement = adapArray[i].measureByteSize() / (1024.0 * 1024.0 * 1024.0); //GBs
        RAMHoursIncrement *= (timeIncrement / 3600.0); //Hours
        RAMHours += RAMHoursIncrement;
        }
        lastEvaluateStartTime = evaluateTime;
        Measurement[] modelMeasurements = new Measurement[]{
                                           new Measurement(
                                           "learning evaluation instances",
                                           instancesProcessed),
                                           new Measurement(
                                           "evaluation time ("
                                           + (preciseCPUTiming ? "cpu "
                                           : "") + "seconds)",
                                           time),
                                           new Measurement(
                                           "model cost (RAM-Hours)",
                                           RAMHours)
                                        };
        List<Measurement> measurementList = new LinkedList<>();
        if (modelMeasurements != null) {
            measurementList.addAll(Arrays.asList(modelMeasurements));
        }
        // add average of sub-model measurements
        if ((windArray != null) && (windArray.length > 0)) {
            List<Measurement[]> subMeasurements = new LinkedList<>();
            for (WindowClassificationPerformanceEvaluator subEvaluator : windArray) {
                if (subEvaluator != null) {
                   subMeasurements.add(subEvaluator.getPerformanceMeasurements());
                }
            }
            Measurement[] avgMeasurements = Measurement.averageMeasurements(subMeasurements.toArray(new Measurement[subMeasurements.size()][]));
            measurementList.addAll(Arrays.asList(avgMeasurements));
        }
           learningCurve.insertEntry(new LearningEvaluation(measurementList.toArray(new Measurement[measurementList.size()])));
        if (immediateResultStream != null) {
            if (firstDump) {
                immediateResultStream.println(learningCurve.headerToString());
                firstDump = false;
            }
            immediateResultStream.print(learnerString + "," + streamString + ","+ Integer.toString(randomSeed) +"," );
            immediateResultStream.println(learningCurve.entryToString(learningCurve.numEntries() - 1));
            immediateResultStream.flush();
        }
        if (firstPrint) {
           System.out.println(learnerString + "," + streamString + ", randomSeed = "+ Integer.toString(randomSeed) +".");
           System.out.print(table.headerLine(learningCurve.headerToString()));
           firstPrint = false;
        }
        System.out.print(table.bodyLine(learningCurve.entryToString(learningCurve.numEntries() - 1)));
        System.out.flush();
    }
}
double time = TimingUtils.nanoTimeToSeconds(TimingUtils.getNanoCPUTimeOfCurrentThread()- evaluateStartTime);
System.out.println(instancesProcessed + " instances processed in " + time + " seconds.");
if (immediateResultStream != null) {
    immediateResultStream.close();
}
